In [1]:
import urllib2
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
tokenizer = RegexpTokenizer(r'\w+')
import sys
from __future__ import division
import pandas as pd
import pyLDAvis
import pyLDAvis.graphlab
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import pyLDAvis.gensim

In [2]:
#Create empty lists for each property
W_TSQ = []
W_USQ = []
W_LA1 = []
W_LA2 = []
W_SF = []
W_MI = []
W_CH = []

Wes_TSQ = []
Wes_GCT = []
Wes_LA1 = []
Wes_LA2 = []
Wes_SF = []
Wes_MI = []
Wes_CH = []

Mar_TSQ = []
Mar_GCT = []
Mar_LA1 = []
Mar_LA2 = []
Mar_SF = []
Mar_MI = []
Mar_CH = []

Hy_TSQ = []
Hy_GCT = []
Hy_LA1 = []
Hy_LA2 = []
Hy_SF = []
Hy_MI = []
Hy_CH = []

Hi_TSQ = []
Hi_GCT = []
Hi_LA1 = []
Hi_LA2 = []
Hi_SF = []
Hi_MI = []
Hi_CH = []

In [3]:
#Parse excel file containing the list of page 1 review results, place in list
xls = pd.ExcelFile('/Users/jessica/Documents/Capstone/Review URLs.xlsx')
df3 = xls.parse('TripA')
#Create list for first page results
url_list_p1 = list(df3.T1)
#Create list for second page results on
url_list_p2 = list(df3.T2)

In [4]:
#Create empty list of lists for the review text and headers
TA_review_text=[W_TSQ, W_USQ, W_LA1, W_LA2, W_SF, W_MI, W_CH, Wes_TSQ, Wes_GCT, Wes_LA1, Wes_LA2, Wes_SF, Wes_MI, Wes_CH, Mar_TSQ, Mar_GCT, Mar_LA1, Mar_LA2, Mar_SF, Mar_MI, Mar_CH, Hy_TSQ, Hy_GCT, Hy_LA1, Hy_LA2, Hy_SF, Hy_MI, Hy_CH, Hi_TSQ, Hi_GCT, Hi_LA1, Hi_LA2, Hi_SF, Hi_MI, Hi_CH]              
TA_review_header=[W_TSQ, W_USQ, W_LA1, W_LA2, W_SF, W_MI, W_CH, Wes_TSQ, Wes_GCT, Wes_LA1, Wes_LA2, Wes_SF, Wes_MI, Wes_CH, Mar_TSQ, Mar_GCT, Mar_LA1, Mar_LA2, Mar_SF, Mar_MI, Mar_CH, Hy_TSQ, Hy_GCT, Hy_LA1, Hy_LA2, Hy_SF, Hy_MI, Hy_CH, Hi_TSQ, Hi_GCT, Hi_LA1, Hi_LA2, Hi_SF, Hi_MI, Hi_CH] 

#Loop through the TripA page 1 urls and add the text and header to its corresponding list within lists
for i in range(len(url_list_p1)):
    url1_page= urllib2.urlopen(url_list_p1[i])
    soup1 = BeautifulSoup(url1_page, 'html.parser')
    TA_review_text[i]=list((soup1.find_all('p',attrs={'class': 'partial_entry'})))
    TA_review_header[i]=list((soup1.find_all('span',attrs={'class': 'noQuotes'})))

In [7]:
TA_review_text[0][0]

<p class="partial_entry">well it's not the first time i stayed here it's my second, i must say this location location location Hotel has the Best location in Times square but i must say this My room 4607 is kind of beat up it's a corner room view...<span class="taLnk ulBlueLinks" onclick="ta.prwidgets.call('handlers.clickExpand',event,this);">More</span></p>

In [9]:
#Loop through the review page results of the urls and add the text and header via "extend" to its corresponding list within lists
for i in range(len(url_list_p2)):
    for j in range(10,100,10):
        j = str(j)
        #print(url_list_p2[i].replace('+j+',j))
        url_page= urllib2.urlopen(url_list_p2[i].replace('+j+',j))
        soup = BeautifulSoup(url_page, 'html.parser')
        TA_review_text[i].extend(list(soup.find_all('p',attrs={'class': 'partial_entry'}))) 
        TA_review_header[i].extend(list(soup.find_all('span',attrs={'class': 'noQuotes'})))

In [11]:
#Create dataframes for text and headers
TA_review_text_pd = pd.DataFrame(data=TA_review_text)
TA_review_header_pd = pd.DataFrame(data=TA_review_header)
TA_review_header_pd.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,"<span class=""noQuotes"">quick little get away <...","<span class=""noQuotes"">WE LOVE THIS HOTEL!!!!!...","<span class=""noQuotes"">Amazing hotel</span>","<span class=""noQuotes"">Request Denied </span>","<span class=""noQuotes"">Good rooms in the heart...","<span class=""noQuotes"">Great Location average ...","<span class=""noQuotes"">Great hotel </span>","<span class=""noQuotes"">Good value if you want ...","<span class=""noQuotes"">Impromptu visit to the ...","<span class=""noQuotes"">Pleasure</span>",...,"<span class=""noQuotes"">Great place to stay in ...","<span class=""noQuotes"">More Like a Club</span>","<span class=""noQuotes"">Not for kids</span>","<span class=""noQuotes"">Stylish modern hotel in...","<span class=""noQuotes"">Last minute NYC trip</s...","<span class=""noQuotes"">customer Service has be...","<span class=""noQuotes"">Love W Concept </span>","<span class=""noQuotes"">Great staff!!!!!</span>","<span class=""noQuotes"">4-8 Inches</span>","<span class=""noQuotes"">One of my favourite Hot..."
1,"<span class=""noQuotes"">Great Location</span>","<span class=""noQuotes"">Consistently good</span>","<span class=""noQuotes"">Hip, cool, fashionable,...","<span class=""noQuotes"">Good choice </span>","<span class=""noQuotes"">Stay Here!</span>","<span class=""noQuotes"">Always a great place</s...","<span class=""noQuotes"">New York W Union Square...","<span class=""noQuotes"">Great Stay in NYC</span>","<span class=""noQuotes"">Funky</span>","<span class=""noQuotes"">Recent Stay at W Union ...",...,"<span class=""noQuotes"">Great hotel, convenient...","<span class=""noQuotes"">Favorite hotel in NYC</...","<span class=""noQuotes"">Another Great Stay</span>","<span class=""noQuotes"">Great location in NYC</...","<span class=""noQuotes"">Great stay!</span>","<span class=""noQuotes"">Wonderful Visit</span>","<span class=""noQuotes"">Personal service at its...","<span class=""noQuotes"">Spacious accomodations<...","<span class=""noQuotes"">Fashionable enough with...","<span class=""noQuotes"">Great service</span>"
2,"<span class=""noQuotes"">Clean &amp; Tidy, but l...","<span class=""noQuotes"">Lovely Stay</span>","<span class=""noQuotes"">Convenient to Pantages<...","<span class=""noQuotes"">A week at the W Hotel</...","<span class=""noQuotes"">PAYMENT FOR INCIDENTALS...","<span class=""noQuotes"">where the beautiful poe...","<span class=""noQuotes"">excellent for business<...","<span class=""noQuotes"">Below average</span>","<span class=""noQuotes"">W Insider Chris Nelson<...","<span class=""noQuotes"">An Escape in LA</span>",...,"<span class=""noQuotes"">Great downtown hotel</s...","<span class=""noQuotes"">Night @W &amp; Anthony ...","<span class=""noQuotes"">Keeps getting Better!!<...","<span class=""noQuotes"">Great Location &amp; Se...","<span class=""noQuotes"">Modern, gadget keen &am...","<span class=""noQuotes"">Honeymooners at the W</...","<span class=""noQuotes"">Offbeat, very ""Hollywoo...","<span class=""noQuotes"">Not a Family Place!</span>","<span class=""noQuotes"">Excellent</span>","<span class=""noQuotes"">Hip in Hollywood</span>"
3,"<span class=""noQuotes"">Decent hotel, indecent ...","<span class=""noQuotes"">Nice hotel close to UCL...","<span class=""noQuotes"">Stay here when you go t...","<span class=""noQuotes"">Great Location, Great S...","<span class=""noQuotes"">Wonderful Stay</span>","<span class=""noQuotes"">Unique hotel</span>","<span class=""noQuotes"">Great and friendly serv...","<span class=""noQuotes"">Great location</span>","<span class=""noQuotes"">Nice hotel, nice rooms,...","<span class=""noQuotes"">Disapointing</span>",...,"<span class=""noQuotes"">Nice Hotel</span>","<span class=""noQuotes"">Spoiled for My Birthday...","<span class=""noQuotes"">RESHANNA IS THE BEST</s...","<span class=""noQuotes"">The W brand needs an up...","<span class=""noQuotes"">Birthday Bash</span>","<span class=""noQuotes"">Great location, elegant...

In [12]:
#Remember to update the range to be the number of columns in the df based on how many review page results I loop through

#Remove tags from within each review for review text
for i in range(100):
    TA_review_text_pd[i] = TA_review_text_pd[i].astype(str).map(lambda x: x.lstrip('<p class="partial_entry">'))
    TA_review_text_pd[i] = TA_review_text_pd[i].astype(str).map(lambda x: x.lstrip('<span class="noQuotes">'))
    TA_review_text_pd[i] = TA_review_text_pd[i].astype(str).map(lambda x: x.rstrip('/span>'))
    TA_review_text_pd[i] = TA_review_text_pd[i].astype(str).map(lambda x: x.rstrip('<'))
    TA_review_text_pd[i] = TA_review_text_pd[i].astype(str).map(lambda x: x.rstrip('/p>'))
    TA_review_text_pd[i] = TA_review_text_pd[i].astype(str).map(lambda x: x.rstrip('<'))
    TA_review_text_pd[i] = TA_review_text_pd[i].astype(str).map(lambda x: x.rstrip('<span class="taLnk ulBlueLinks" onclick="ta.prwidgets.call(\'handlers.clickExpand\',event,this);">More</span'))

#Remove tags from within each review for review headers
for i in range(50):
    TA_review_header_pd[i] = TA_review_header_pd[i].astype(str).map(lambda x: x.lstrip('<p class="partial_entry">'))
    TA_review_header_pd[i] = TA_review_header_pd[i].astype(str).map(lambda x: x.lstrip('<span class="noQuotes">'))
    TA_review_header_pd[i] = TA_review_header_pd[i].astype(str).map(lambda x: x.rstrip('/span>'))
    TA_review_header_pd[i] = TA_review_header_pd[i].astype(str).map(lambda x: x.rstrip('<'))
    TA_review_header_pd[i] = TA_review_header_pd[i].astype(str).map(lambda x: x.rstrip('/p>'))
    TA_review_header_pd[i] = TA_review_header_pd[i].astype(str).map(lambda x: x.rstrip('<'))
    
TA_review_text_pd1 = TA_review_text_pd
TA_review_header_pd1 = TA_review_header_pd

In [13]:
TA_review_text_pd1

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,well it's not the first time i stayed here it'...,Me and my partner recently visited New York fo...,Hi JHLucyG!\nThank you for your wonderful revi...,Great hotel and even better location. The room...,Patrick helped us with our suitcases. Adam bro...,"Typical ""W"" hotel: lively, modern, good bar. R...",The hotel has a great location with descent se...,Came to NYC for the weekend for my daughter's ...,I doubt you are going to find a better price f...,Loved this hotel. I checked in early and was u...,...,N,N,N,N,N,N,N,N,N,N
1,This was my third time staying at this hotel. ...,I stay at this property at least 1-2 times a y...,"Hello Michael Y,\nIt is always a joy to welcom...",Starting at the entrance the doormen are gener...,We went back for a weekend in the city because...,"Hello Steve S,\nGreetings from the old neighbo...",Really love this location. My corner room was ...,"Hello Mlyf65,\nYour outstanding recommendation...",Every year I spend about 3 weekends at the W H...,"Hello Jim P,\nWarm greetings from your family ...",...,Though a reasonably regular visitor to the W I...,"Hello PixieWife,\nWe enjoyed welcoming you bac...",Stayed here for a wedding and once I got over ...,"Hello James C,\nWe love celebrations at W Unio...",Stayed here on first weekend visit to NYC. Cer...,"Hello Cinebriate,\nWarm greetings from W Union...",After reading some recent reviews on trip advi...,"Hello Aleen B,\nWarm greetings from your new f...",N,N
2,"Stayed here for 2 nights at the end of May, ar...","Hello Aus_Trav_88,\nThank you for sharing comm...",This is a great hotel if you're staying in Hol...,"Hello Ellen28,\nThank you for taking the time ...",I took my 14 year old granddaughter to the Pan...,"Hello stainedglasslass,\nThank you for sharing...",I didn't find the staff very approachable duri...,"Hello fancypantsdancer2009,\nThank you for tak...",Te hotel gives you a very hard time if your cr...,"Dear Raju H,\nThank you for staying with, we a...",...,N,N,N,N,N,N,N,N,N,N
3,I stay at W and Starwood a lot - 80 nights in ...,"Hello, N K,\nThank you for taking the time to ...",We stayed here this past weekend because our d...,"Hello Evi_02,\nThanks for taking the time to r...",The W Los Angeles is in a great location for a...,I really enjoyed how close the W was to the UC...,"Nestled off in a residential area of Westwood,...","My feedback is long overdue, I have stayed at ...","Hello CrankyCracker,\nThanks for taking the ti...",I stay at this hotel regularly for work and I ...,...,"Hello Mark,\nThank you for your wonderful feed...",I must say the W in Beverley Hills is just gre...,"Hello Mark,\nThank you for your wonderful feed...",My stay at the W Los Angeles - West Beverly Hi...,"Hello Zach,\nThank you for the feedback on you...",A delightful overnight stay for our extended f...,"Hello SirWanderer,\nThank you for your wonderf...",N,N,N
4,The hotel is well located and very well appoin...,Hotel is located in the South of Market area a...,"Hello Benjamin,\nThank you so very much for yo...","The hotel is gorgeous - from the lobby, to the...",The perfect location if you're keen to explore...,I had a nice corner room which was nice and co...,I was incorrectly charged by the hotel:\nFirst...,Stayed at the W on a Saturday night which is t...,My girlfriend has serious status with and is a...,"Jeremy,\nWe truly appreciate your amazing note...",...,derstand the hotel has been redecorated. it's ...,"Hello Sarah,\nThank you for your candid feedba...","There are some nice features, but the negative...",Hello guyfromroseville!\nThank you very much f...,N,N,N,N,N,N
5,We had such a nice stay at the W Miami. We had...,I want to be positive about our experience her...,"Dear ukinmiami,\nThank you for sharing your re...",Over the years I have been loyal to the W Hote...,"Dear Gonefishing101,\nThank you for taking the...",We stayed here prior to a cruise. Well-maintai...,was there to get a mini staycation an

In [14]:
#Break the dataframe into a series for each property
W_TSQ_A = pd.Series(TA_review_text_pd1.ix[0,:].unique())
W_TSQ_B = pd.Series(TA_review_header_pd1.ix[0,:].unique())
W_TSQ = pd.concat([W_TSQ_A, W_TSQ_B], axis=0, ignore_index=True)

W_USQ_A = pd.Series(TA_review_text_pd1.ix[1,:].unique())
W_USQ_B = pd.Series(TA_review_header_pd1.ix[1,:].unique())
W_USQ = pd.concat([W_USQ_A, W_USQ_B], axis=0, ignore_index=True)

W_LA1_A = pd.Series(TA_review_text_pd1.ix[2,:].unique())
W_LA1_B = pd.Series(TA_review_header_pd1.ix[2,:].unique())
W_LA1 = pd.concat([W_LA1_A, W_LA1_B], axis=0, ignore_index=True)

W_LA2_A = pd.Series(TA_review_text_pd1.ix[3,:].unique())
W_LA2_B = pd.Series(TA_review_header_pd1.ix[3,:].unique())
W_LA2 = pd.concat([W_LA2_A, W_LA2_B], axis=0, ignore_index=True)

W_SF_A = pd.Series(TA_review_text_pd1.ix[4,:].unique())
W_SF_B = pd.Series(TA_review_header_pd1.ix[4,:].unique())
W_SF = pd.concat([W_SF_A, W_SF_B], axis=0, ignore_index=True)

W_MI_A = pd.Series(TA_review_text_pd1.ix[5,:].unique())
W_MI_B = pd.Series(TA_review_header_pd1.ix[5,:].unique())
W_MI = pd.concat([W_MI_A, W_MI_B], axis=0, ignore_index=True)

W_CH_A = pd.Series(TA_review_text_pd1.ix[6,:].unique())
W_CH_B = pd.Series(TA_review_header_pd1.ix[6,:].unique())
W_CH = pd.concat([W_CH_A, W_CH_B], axis=0, ignore_index=True)

Wes_TSQ_A = pd.Series(TA_review_text_pd1.ix[7,:].unique())
Wes_TSQ_B = pd.Series(TA_review_header_pd1.ix[7,:].unique())
Wes_TSQ = pd.concat([Wes_TSQ_A, Wes_TSQ_B], axis=0, ignore_index=True)

Wes_GCT_A = pd.Series(TA_review_text_pd1.ix[8,:].unique())
Wes_GCT_B = pd.Series(TA_review_header_pd1.ix[8,:].unique())
Wes_GCT = pd.concat([Wes_GCT_A, Wes_GCT_B], axis=0, ignore_index=True)

Wes_LA1_A = pd.Series(TA_review_text_pd1.ix[9,:].unique())
Wes_LA1_B = pd.Series(TA_review_header_pd1.ix[9,:].unique())
Wes_LA1 = pd.concat([Wes_LA1_A, Wes_LA1_B], axis=0, ignore_index=True)

Wes_LA2_A = pd.Series(TA_review_text_pd1.ix[10,:].unique())
Wes_LA2_B = pd.Series(TA_review_header_pd1.ix[10,:].unique())
Wes_LA2 = pd.concat([Wes_LA2_A, Wes_LA2_B], axis=0, ignore_index=True)

Wes_SF_A = pd.Series(TA_review_text_pd1.ix[11,:].unique())
Wes_SF_B = pd.Series(TA_review_header_pd1.ix[11,:].unique())
Wes_SF = pd.concat([Wes_SF_A, Wes_SF_B], axis=0, ignore_index=True)

Wes_MI_A = pd.Series(TA_review_text_pd1.ix[12,:].unique())
Wes_MI_B = pd.Series(TA_review_header_pd1.ix[12,:].unique())
Wes_MI = pd.concat([Wes_MI_A, Wes_MI_B], axis=0, ignore_index=True)

Wes_CH_A = pd.Series(TA_review_text_pd1.ix[13,:].unique())
Wes_CH_B = pd.Series(TA_review_header_pd1.ix[13,:].unique())
Wes_CH = pd.concat([Wes_CH_A, Wes_CH_B], axis=0, ignore_index=True)

Mar_TSQ_A = pd.Series(TA_review_text_pd1.ix[14,:].unique())
Mar_TSQ_B = pd.Series(TA_review_header_pd1.ix[14,:].unique())
Mar_TSQ = pd.concat([Mar_TSQ_A, Mar_TSQ_B], axis=0, ignore_index=True)

Mar_GCT_A = pd.Series(TA_review_text_pd1.ix[15,:].unique())
Mar_GCT_B = pd.Series(TA_review_header_pd1.ix[15,:].unique())
Mar_GCT = pd.concat([Mar_GCT_A, Mar_GCT_B], axis=0, ignore_index=True)

Mar_LA1_A = pd.Series(TA_review_text_pd1.ix[16,:].unique())
Mar_LA1_B = pd.Series(TA_review_header_pd1.ix[16,:].unique())
Mar_LA1 = pd.concat([Mar_LA1_A, Mar_LA1_B], axis=0, ignore_index=True)

Mar_LA2_A = pd.Series(TA_review_text_pd1.ix[17,:].unique())
Mar_LA2_B = pd.Series(TA_review_header_pd1.ix[17,:].unique())
Mar_LA2 = pd.concat([Mar_LA2_A, Mar_LA2_B], axis=0, ignore_index=True)

Mar_SF_A = pd.Series(TA_review_text_pd1.ix[18,:].unique())
Mar_SF_B = pd.Series(TA_review_header_pd1.ix[18,:].unique())
Mar_SF = pd.concat([Mar_SF_A, Mar_SF_B], axis=0, ignore_index=True)

Mar_MI_A = pd.Series(TA_review_text_pd1.ix[19,:].unique())
Mar_MI_B = pd.Series(TA_review_header_pd1.ix[19,:].unique())
Mar_MI = pd.concat([Mar_MI_A, Mar_MI_B], axis=0, ignore_index=True)

Mar_CH_A = pd.Series(TA_review_text_pd1.ix[20,:].unique())
Mar_CH_B = pd.Series(TA_review_header_pd1.ix[20,:].unique())
Mar_CH = pd.concat([Mar_CH_A, Mar_CH_B], axis=0, ignore_index=True)

Hy_TSQ_A = pd.Series(TA_review_text_pd1.ix[21,:].unique())
Hy_TSQ_B = pd.Series(TA_review_header_pd1.ix[21,:].unique())
Hy_TSQ = pd.concat([Hy_TSQ_A, Hy_TSQ_B], axis=0, ignore_index=True)

Hy_GCT_A = pd.Series(TA_review_text_pd1.ix[22,:].unique())
Hy_GCT_B = pd.Series(TA_review_header_pd1.ix[22,:].unique())
Hy_GCT = pd.concat([Hy_GCT_A, Hy_GCT_B], axis=0, ignore_index=True)

Hy_LA1_A = pd.Series(TA_review_text_pd1.ix[23,:].unique())
Hy_LA1_B = pd.Series(TA_review_header_pd1.ix[23,:].unique())
Hy_LA1 = pd.concat([Hy_LA1_A, Hy_LA1_B], axis=0, ignore_index=True)

Hy_LA2_A = pd.Series(TA_review_text_pd1.ix[24,:].unique())
Hy_LA2_B = pd.Series(TA_review_header_pd1.ix[24,:].unique())
Hy_LA2 = pd.concat([Hy_LA2_A, Hy_LA2_B], axis=0, ignore_index=True)

Hy_SF_A = pd.Series(TA_review_text_pd1.ix[25,:].unique())
Hy_SF_B = pd.Series(TA_review_header_pd1.ix[25,:].unique())
Hy_SF = pd.concat([Hy_SF_A, Hy_SF_B], axis=0, ignore_index=True)

Hy_MI_A = pd.Series(TA_review_text_pd1.ix[26,:].unique())
Hy_MI_B = pd.Series(TA_review_header_pd1.ix[26,:].unique())
Hy_MI = pd.concat([Hy_MI_A, Hy_MI_B], axis=0, ignore_index=True)

Hy_CH_A = pd.Series(TA_review_text_pd1.ix[27,:].unique())
Hy_CH_B = pd.Series(TA_review_header_pd1.ix[27,:].unique())
Hy_CH = pd.concat([Hy_CH_A, Hy_CH_B], axis=0, ignore_index=True)

Hi_TSQ_A = pd.Series(TA_review_text_pd1.ix[28,:].unique())
Hi_TSQ_B = pd.Series(TA_review_header_pd1.ix[28,:].unique())
Hi_TSQ = pd.concat([Hi_TSQ_A, Hi_TSQ_B], axis=0, ignore_index=True)

Hi_GCT_A = pd.Series(TA_review_text_pd1.ix[29,:].unique())
Hi_GCT_B = pd.Series(TA_review_header_pd1.ix[29,:].unique())
Hi_GCT = pd.concat([Hi_TSQ_A, Hi_TSQ_B], axis=0, ignore_index=True)

Hi_LA1_A = pd.Series(TA_review_text_pd1.ix[30,:].unique())
Hi_LA1_B = pd.Series(TA_review_header_pd1.ix[30,:].unique())
Hi_LA1 = pd.concat([Hi_LA1_A, Hi_LA1_B], axis=0, ignore_index=True)

Hi_LA2_A = pd.Series(TA_review_text_pd1.ix[31,:].unique())
Hi_LA2_B = pd.Series(TA_review_header_pd1.ix[31,:].unique())
Hi_LA2 = pd.concat([Hi_LA2_A, Hi_LA2_B], axis=0, ignore_index=True)

Hi_SF_A = pd.Series(TA_review_text_pd1.ix[32,:].unique())
Hi_SF_B = pd.Series(TA_review_header_pd1.ix[32,:].unique())
Hi_SF = pd.concat([Hi_SF_A, Hi_SF_B], axis=0, ignore_index=True)

Hi_MI_A = pd.Series(TA_review_text_pd1.ix[33,:].unique())
Hi_MI_B = pd.Series(TA_review_header_pd1.ix[33,:].unique())
Hi_MI = pd.concat([Hi_MI_A, Hi_MI_B], axis=0, ignore_index=True)

Hi_CH_A = pd.Series(TA_review_text_pd1.ix[34,:].unique())
Hi_CH_B = pd.Series(TA_review_header_pd1.ix[34,:].unique())
Hi_CH = pd.concat([Hi_CH_A, Hi_CH_B], axis=0, ignore_index=True)

In [15]:
#Check text for a property
Hi_CH[5]

'From guest checkin, the room, the amenities, and the service--this property does nothing but shine bright. The Hilton Chicago has the very best of everything, including stylish and generous guest rooms, executive floor services, and the best service personnel to be found anywhere.\nMaking freq'

In [16]:
#Filter out replies from the hotel. Won't be able to filter out names
W_TSQ = filter(lambda s: 'Hello' not in str(s), W_TSQ)
W_TSQ = filter(lambda s: 'Welcome' not in str(s), W_TSQ)
W_TSQ = filter(lambda s: 'Hi ' not in str(s), W_TSQ)
W_TSQ = filter(lambda s: 'Dear' not in str(s), W_TSQ)
W_TSQ = filter(lambda s: 'None' not in str(s), W_TSQ)

W_USQ = filter(lambda s: 'Hello' not in str(s), W_USQ)
W_USQ = filter(lambda s: 'Welcome' not in str(s), W_USQ)
W_USQ = filter(lambda s: 'Hi ' not in str(s), W_USQ)
W_USQ = filter(lambda s: 'Dear' not in str(s), W_USQ)
W_USQ = filter(lambda s: 'None' not in str(s), W_USQ)

W_LA1 = filter(lambda s: 'Hello' not in str(s), W_LA1)
W_LA1 = filter(lambda s: 'Welcome' not in str(s), W_LA1)
W_LA1 = filter(lambda s: 'Hi ' not in str(s), W_LA1)
W_LA1 = filter(lambda s: 'Dear' not in str(s), W_LA1)
W_LA1 = filter(lambda s: 'None' not in str(s), W_LA1)

W_LA2 = filter(lambda s: 'Hello' not in str(s), W_LA2)
W_LA2 = filter(lambda s: 'Welcome' not in str(s), W_LA2)
W_LA2 = filter(lambda s: 'Hi ' not in str(s), W_LA2)
W_LA2 = filter(lambda s: 'Dear' not in str(s), W_LA2)
W_LA2 = filter(lambda s: 'None' not in str(s), W_LA2)

W_SF = filter(lambda s: 'Hello' not in str(s), W_SF)
W_SF = filter(lambda s: 'Welcome ' not in str(s), W_SF)
W_SF = filter(lambda s: 'Hi ' not in str(s), W_SF)
W_SF = filter(lambda s: 'Dear ' not in str(s), W_SF)
W_SF = filter(lambda s: 'None' not in str(s), W_SF)
W_SF

W_MI = filter(lambda s: 'Hello' not in str(s), W_MI)
W_MI = filter(lambda s: 'Welcome ' not in str(s), W_MI)
W_MI = filter(lambda s: 'Hi ' not in str(s), W_MI)
W_MI = filter(lambda s: 'Dear ' not in str(s), W_MI)
W_MI = filter(lambda s: 'None' not in str(s), W_MI)

W_CH = filter(lambda s: 'Hello' not in str(s), W_CH)
W_CH = filter(lambda s: 'Welcome ' not in str(s), W_CH)
W_CH = filter(lambda s: 'Hi ' not in str(s), W_CH)
W_CH = filter(lambda s: 'Dear ' not in str(s), W_CH)
W_CH = filter(lambda s: 'None' not in str(s), W_CH)

Wes_TSQ = filter(lambda s: 'Hello' not in str(s), Wes_TSQ)
Wes_TSQ = filter(lambda s: 'Welcome ' not in str(s), Wes_TSQ)
Wes_TSQ = filter(lambda s: 'Hi ' not in str(s), Wes_TSQ)
Wes_TSQ = filter(lambda s: 'Dear ' not in str(s), Wes_TSQ)
Wes_TSQ = filter(lambda s: 'None' not in str(s), Wes_TSQ)

Wes_GCT = filter(lambda s: 'Hello' not in str(s), Wes_GCT)
Wes_GCT = filter(lambda s: 'Welcome ' not in str(s), Wes_GCT)
Wes_GCT = filter(lambda s: 'Hi ' not in str(s), Wes_GCT)
Wes_GCT = filter(lambda s: 'Dear ' not in str(s), Wes_GCT)
Wes_GCT = filter(lambda s: 'None' not in str(s), Wes_GCT)

Wes_LA1 = filter(lambda s: 'Hello' not in str(s), Wes_LA1)
Wes_LA1 = filter(lambda s: 'Welcome ' not in str(s), Wes_LA1)
Wes_LA1 = filter(lambda s: 'Hi ' not in str(s), Wes_LA1)
Wes_LA1 = filter(lambda s: 'Dear ' not in str(s), Wes_LA1)
Wes_LA1 = filter(lambda s: 'None' not in str(s), Wes_LA1)

Wes_LA2 = filter(lambda s: 'Hello' not in str(s), Wes_LA2)
Wes_LA2 = filter(lambda s: 'Welcome ' not in str(s), Wes_LA2)
Wes_LA2 = filter(lambda s: 'Hi ' not in str(s), Wes_LA2)
Wes_LA2 = filter(lambda s: 'Dear ' not in str(s), Wes_LA2)
Wes_LA2 = filter(lambda s: 'None' not in str(s), Wes_LA2)

Wes_SF = filter(lambda s: 'Hello' not in str(s), Wes_SF)
Wes_SF = filter(lambda s: 'Welcome ' not in str(s), Wes_SF)
Wes_SF = filter(lambda s: 'Hi ' not in str(s), Wes_SF)
Wes_SF = filter(lambda s: 'Dear ' not in str(s), Wes_SF)
Wes_SF = filter(lambda s: 'None' not in str(s), Wes_SF)

Wes_MI = filter(lambda s: 'Hello' not in str(s), Wes_MI)
Wes_MI = filter(lambda s: 'Welcome ' not in str(s), Wes_MI)
Wes_MI = filter(lambda s: 'Hi ' not in str(s), Wes_MI)
Wes_MI = filter(lambda s: 'Dear ' not in str(s), Wes_MI)
Wes_MI = filter(lambda s: 'None' not in str(s), Wes_MI)

Wes_CH = filter(lambda s: 'Hello' not in str(s), Wes_CH)
Wes_CH = filter(lambda s: 'Welcome ' not in str(s), Wes_CH)
Wes_CH = filter(lambda s: 'Hi ' not in str(s), Wes_CH)
Wes_CH = filter(lambda s: 'Dear ' not in str(s), Wes_CH)
Wes_CH = filter(lambda s: 'None' not in str(s), Wes_CH)

Mar_TSQ = filter(lambda s: 'Hello' not in str(s), Mar_TSQ)
Mar_TSQ = filter(lambda s: 'Welcome ' not in str(s), Mar_TSQ)
Mar_TSQ = filter(lambda s: 'Hi ' not in str(s), Mar_TSQ)
Mar_TSQ = filter(lambda s: 'Dear ' not in str(s), Mar_TSQ)
Mar_TSQ = filter(lambda s: 'None' not in str(s), Mar_TSQ)

Mar_GCT = filter(lambda s: 'Hello' not in str(s), Mar_GCT)
Mar_GCT = filter(lambda s: 'Welcome ' not in str(s), Mar_GCT)
Mar_GCT = filter(lambda s: 'Hi ' not in str(s), Mar_GCT)
Mar_GCT = filter(lambda s: 'Dear ' not in str(s), Mar_GCT)
Mar_GCT = filter(lambda s: 'None' not in str(s), Mar_GCT)

Mar_LA1 = filter(lambda s: 'Hello' not in str(s), Mar_LA1)
Mar_LA1 = filter(lambda s: 'Welcome ' not in str(s), Mar_LA1)
Mar_LA1 = filter(lambda s: 'Hi ' not in str(s), Mar_LA1)
Mar_LA1 = filter(lambda s: 'Dear ' not in str(s), Mar_LA1)
Mar_LA1 = filter(lambda s: 'None' not in str(s), Mar_LA1)

Mar_LA2 = filter(lambda s: 'Hello' not in str(s), Mar_LA2)
Mar_LA2 = filter(lambda s: 'Welcome ' not in str(s), Mar_LA2)
Mar_LA2 = filter(lambda s: 'Hi ' not in str(s), Mar_LA2)
Mar_LA2 = filter(lambda s: 'Dear ' not in str(s), Mar_LA2)
Mar_LA2 = filter(lambda s: 'None' not in str(s), Mar_LA2)

Mar_SF = filter(lambda s: 'Hello' not in str(s), Mar_SF)
Mar_SF = filter(lambda s: 'Welcome ' not in str(s), Mar_SF)
Mar_SF = filter(lambda s: 'Hi ' not in str(s), Mar_SF)
Mar_SF = filter(lambda s: 'Dear ' not in str(s), Mar_SF)
Mar_SF = filter(lambda s: 'None' not in str(s), Mar_SF)

Mar_MI = filter(lambda s: 'Hello' not in str(s), Mar_MI)
Mar_MI = filter(lambda s: 'Welcome ' not in str(s), Mar_MI)
Mar_MI = filter(lambda s: 'Hi ' not in str(s), Mar_MI)
Mar_MI = filter(lambda s: 'Dear ' not in str(s), Mar_MI)
Mar_MI = filter(lambda s: 'None' not in str(s), Mar_MI)

Mar_CH = filter(lambda s: 'Hello' not in str(s), Mar_CH)
Mar_CH = filter(lambda s: 'Welcome ' not in str(s), Mar_CH)
Mar_CH = filter(lambda s: 'Hi ' not in str(s), Mar_CH)
Mar_CH = filter(lambda s: 'Dear ' not in str(s), Mar_CH)
Mar_CH = filter(lambda s: 'None' not in str(s), Mar_CH)

Hy_TSQ = filter(lambda s: 'Hello' not in str(s), Hy_TSQ)
Hy_TSQ = filter(lambda s: 'Welcome ' not in str(s), Hy_TSQ)
Hy_TSQ = filter(lambda s: 'Hi ' not in str(s), Hy_TSQ)
Hy_TSQ = filter(lambda s: 'Dear ' not in str(s), Hy_TSQ)
Hy_TSQ = filter(lambda s: 'None' not in str(s), Hy_TSQ)

Hy_GCT = filter(lambda s: 'Hello' not in str(s), Hy_GCT)
Hy_GCT = filter(lambda s: 'Welcome ' not in str(s), Hy_GCT)
Hy_GCT = filter(lambda s: 'Hi ' not in str(s), Hy_GCT)
Hy_GCT = filter(lambda s: 'Dear ' not in str(s), Hy_GCT)
Hy_GCT = filter(lambda s: 'None' not in str(s), Hy_GCT)

Hy_LA1 = filter(lambda s: 'Hello' not in str(s), Hy_LA1)
Hy_LA1 = filter(lambda s: 'Welcome ' not in str(s), Hy_LA1)
Hy_LA1 = filter(lambda s: 'Hi ' not in str(s), Hy_LA1)
Hy_LA1 = filter(lambda s: 'Dear ' not in str(s), Hy_LA1)
Hy_LA1 = filter(lambda s: 'None' not in str(s), Hy_LA1)

Hy_LA2 = filter(lambda s: 'Hello' not in str(s), Hy_LA2)
Hy_LA2 = filter(lambda s: 'Welcome ' not in str(s), Hy_LA2)
Hy_LA2 = filter(lambda s: 'Hi ' not in str(s), Hy_LA2)
Hy_LA2 = filter(lambda s: 'Dear ' not in str(s), Hy_LA2)
Hy_LA2 = filter(lambda s: 'None' not in str(s), Hy_LA2)

Hy_SF = filter(lambda s: 'Hello' not in str(s), Hy_SF)
Hy_SF = filter(lambda s: 'Welcome ' not in str(s), Hy_SF)
Hy_SF = filter(lambda s: 'Hi ' not in str(s), Hy_SF)
Hy_SF = filter(lambda s: 'Dear ' not in str(s), Hy_SF)
Hy_SF = filter(lambda s: 'None' not in str(s), Hy_SF)

Hy_MI = filter(lambda s: 'Hello' not in str(s), Hy_MI)
Hy_MI = filter(lambda s: 'Welcome ' not in str(s), Hy_MI)
Hy_MI = filter(lambda s: 'Hi ' not in str(s), Hy_MI)
Hy_MI = filter(lambda s: 'Dear ' not in str(s), Hy_MI)
Hy_MI = filter(lambda s: 'None' not in str(s), Hy_MI)

Hy_CH = filter(lambda s: 'Hello' not in str(s), Hy_CH)
Hy_CH = filter(lambda s: 'Welcome ' not in str(s), Hy_CH)
Hy_CH = filter(lambda s: 'Hi ' not in str(s), Hy_CH)
Hy_CH = filter(lambda s: 'Dear ' not in str(s), Hy_CH)
Hy_CH = filter(lambda s: 'None' not in str(s), Hy_CH)

Hi_TSQ = filter(lambda s: 'Hello' not in str(s), Hi_TSQ)
Hi_TSQ = filter(lambda s: 'Welcome ' not in str(s), Hi_TSQ)
Hi_TSQ = filter(lambda s: 'Hi ' not in str(s), Hi_TSQ)
Hi_TSQ = filter(lambda s: 'Dear ' not in str(s), Hi_TSQ)
Hi_TSQ = filter(lambda s: 'None' not in str(s), Hi_TSQ)

Hi_GCT = filter(lambda s: 'Hello' not in str(s), Hi_GCT)
Hi_GCT = filter(lambda s: 'Welcome ' not in str(s), Hi_GCT)
Hi_GCT = filter(lambda s: 'Hi ' not in str(s), Hi_GCT)
Hi_GCT = filter(lambda s: 'Dear ' not in str(s), Hi_GCT)
Hi_GCT = filter(lambda s: 'None' not in str(s), Hi_GCT)

Hi_LA1 = filter(lambda s: 'Hello' not in str(s), Hi_LA1)
Hi_LA1 = filter(lambda s: 'Welcome ' not in str(s), Hi_LA1)
Hi_LA1 = filter(lambda s: 'Hi ' not in str(s), Hi_LA1)
Hi_LA1 = filter(lambda s: 'Dear ' not in str(s), Hi_LA1)
Hi_LA1 = filter(lambda s: 'None' not in str(s), Hi_LA1)

Hi_LA2 = filter(lambda s: 'Hello' not in str(s), Hi_LA2)
Hi_LA2 = filter(lambda s: 'Welcome ' not in str(s), Hi_LA2)
Hi_LA2 = filter(lambda s: 'Hi ' not in str(s), Hi_LA2)
Hi_LA2 = filter(lambda s: 'Dear ' not in str(s), Hi_LA2)
Hi_LA2 = filter(lambda s: 'None' not in str(s), Hi_LA2)

Hi_SF = filter(lambda s: 'Hello' not in str(s), Hi_SF)
Hi_SF = filter(lambda s: 'Welcome ' not in str(s), Hi_SF)
Hi_SF = filter(lambda s: 'Hi ' not in str(s), Hi_SF)
Hi_SF = filter(lambda s: 'Dear ' not in str(s), Hi_SF)
Hi_SF = filter(lambda s: 'None' not in str(s), Hi_SF)

Hi_MI = filter(lambda s: 'Hello' not in str(s), Hi_MI)
Hi_MI = filter(lambda s: 'Welcome ' not in str(s), Hi_MI)
Hi_MI = filter(lambda s: 'Hi ' not in str(s), Hi_MI)
Hi_MI = filter(lambda s: 'Dear ' not in str(s), Hi_MI)
Hi_MI = filter(lambda s: 'None' not in str(s), Hi_MI)

Hi_CH = filter(lambda s: 'Hello' not in str(s), Hi_CH)
Hi_CH = filter(lambda s: 'Welcome ' not in str(s), Hi_CH)
Hi_CH = filter(lambda s: 'Hi ' not in str(s), Hi_CH)
Hi_CH = filter(lambda s: 'Dear ' not in str(s), Hi_CH)
Hi_CH = filter(lambda s: 'None' not in str(s), Hi_CH)

In [54]:
#Prepare data for LDA
#create English stop words list
en_stop = set(stopwords.words('english'))

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

en_stop.update((u'hotel',u'locat',u'room',u'stay',u'great',u'rooms',u'w',u'westin',u'hyatt',u'hilton',u'marriott',u'location',u'times',u'good',u'stayed',u'would',u'nice',u'square',
                u'city',u'union',u'grand',u'central',u'hollywood',u'new',u'york',u'los',u'angeles',u'san',u'francisco',u'chicago',u'miami',u'night',u'amp',u'property',u'well',
                u'get',u'nyc',u'sf',u'la',u'fran',u'much',u'instead',u'westwood',u'us',u'time',u'year',u'could'))
en_stop

{u'a',
 u'about',
 u'above',
 u'after',
 u'again',
 u'against',
 u'ain',
 u'all',
 u'am',
 u'amp',
 u'an',
 u'and',
 u'angeles',
 u'any',
 u'are',
 u'aren',
 u'as',
 u'at',
 u'be',
 u'because',
 u'been',
 u'before',
 u'being',
 u'below',
 u'between',
 u'both',
 u'but',
 u'by',
 u'can',
 u'central',
 u'chicago',
 u'city',
 u'couldn',
 u'd',
 u'did',
 u'didn',
 u'do',
 u'does',
 u'doesn',
 u'doing',
 u'don',
 u'down',
 u'during',
 u'each',
 u'few',
 u'for',
 u'fran',
 u'francisco',
 u'from',
 u'further',
 u'get',
 u'good',
 u'grand',
 u'great',
 u'had',
 u'hadn',
 u'has',
 u'hasn',
 u'have',
 u'haven',
 u'having',
 u'he',
 u'her',
 u'here',
 u'hers',
 u'herself',
 u'hilton',
 u'him',
 u'himself',
 u'his',
 u'hollywood',
 u'hotel',
 u'how',
 u'hyatt',
 u'i',
 u'if',
 u'in',
 u'instead',
 u'into',
 u'is',
 u'isn',
 u'it',
 u'its',
 u'itself',
 u'just',
 u'la',
 u'll',
 u'locat',
 u'location',
 u'los',
 u'm',
 u'ma',
 u'marriott',
 u'me',
 u'miami',
 u'mightn',
 u'more',
 u'most',
 u'much',

In [55]:
#Compile documents into a list
W_TSQ_s = ''.join(str(e) for e in W_TSQ)
W_USQ_s = ''.join(str(e) for e in W_USQ)
W_LA1_s = ''.join(str(e) for e in W_LA1)
W_LA2_s = ''.join(str(e) for e in W_LA2)
W_SF_s = ''.join(str(e) for e in W_SF)
W_MI_s = ''.join(str(e) for e in W_MI)
W_CH_s = ''.join(str(e) for e in W_CH)
W_set = [W_TSQ_s, W_USQ_s, W_LA1_s, W_LA2_s, W_SF_s, W_MI_s, W_CH_s] 
W_text = []

Wes_TSQ_s = ''.join(str(e) for e in Wes_TSQ)
Wes_GCT_s = ''.join(str(e) for e in Wes_GCT)
Wes_LA1_s = ''.join(str(e) for e in Wes_LA1)
Wes_LA2_s = ''.join(str(e) for e in Wes_LA2)
Wes_SF_s = ''.join(str(e) for e in Wes_SF)
Wes_MI_s = ''.join(str(e) for e in Wes_MI)
Wes_CH_s = ''.join(str(e) for e in Wes_CH)
Wes_set = [Wes_TSQ, Wes_GCT, Wes_LA1, Wes_LA2, Wes_SF, Wes_MI, Wes_CH] 
Wes_text = []

Mar_TSQ_s = ''.join(str(e) for e in Mar_TSQ)
Mar_GCT_s = ''.join(str(e) for e in Mar_GCT)
Mar_LA1_s = ''.join(str(e) for e in Mar_LA1)
Mar_LA2_s = ''.join(str(e) for e in Mar_LA2)
Mar_SF_s = ''.join(str(e) for e in Mar_SF)
Mar_MI_s = ''.join(str(e) for e in Mar_MI)
Mar_CH_s = ''.join(str(e) for e in Mar_CH)
Mar_set = [Mar_TSQ, Mar_GCT, Mar_LA1, Mar_LA2, Mar_SF, Mar_MI, Mar_CH]
Mar_text = []

Hy_TSQ_s = ''.join(str(e) for e in Hy_TSQ)
Hy_GCT_s = ''.join(str(e) for e in Hy_GCT)
Hy_LA1_s = ''.join(str(e) for e in Hy_LA1)
Hy_LA2_s = ''.join(str(e) for e in Hy_LA2)
Hy_SF_s = ''.join(str(e) for e in Hy_SF)
Hy_MI_s = ''.join(str(e) for e in Hy_MI)
Hy_CH_s = ''.join(str(e) for e in Hy_CH)
Hy_set = [Hy_TSQ, Hy_GCT, Hy_LA1, Hy_LA2, Hy_SF, Hy_MI, Hy_CH]
Hy_text = []

Hi_TSQ_s = ''.join(str(e) for e in Hi_TSQ)
Hi_GCT_s = ''.join(str(e) for e in Hi_GCT)
Hi_LA1_s = ''.join(str(e) for e in Hi_LA1)
Hi_LA2_s = ''.join(str(e) for e in Hi_LA2)
Hi_SF_s = ''.join(str(e) for e in Hi_SF)
Hi_MI_s = ''.join(str(e) for e in Hi_MI)
Hi_CH_s = ''.join(str(e) for e in Hi_CH)
Hi_set = [Hi_TSQ, Hi_GCT, Hi_LA1, Hi_LA2, Hi_SF, Hi_MI, Hi_CH]
Hi_text = []

TripA_set = [W_TSQ_s, W_USQ_s, W_LA1_s, W_LA2_s, W_SF_s, W_MI_s, W_CH_s, Wes_TSQ, Wes_GCT, Wes_LA1, Wes_LA2, 
             Wes_SF, Wes_MI, Wes_CH, Mar_TSQ, Mar_GCT, Mar_LA1, Hy_TSQ, Hy_GCT, Hy_LA1, Hy_LA2, Hy_SF, Hy_MI, 
             Hy_CH, Mar_LA2, Mar_SF, Mar_MI, Mar_CH,Hy_TSQ, Hy_GCT, Hy_LA1, Hy_LA2, Hy_SF, Hy_MI, Hy_CH, Hi_TSQ, Hi_GCT, Hi_LA1, Hi_LA2, Hi_SF, Hi_MI, Hi_CH]
Trip_A_text = []

In [56]:
# loop through W reviews
for i in W_set:    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    # remove stop words from tokens 
    stopped_tokens = [i.decode('utf-8','ignore').encode("utf-8") for i in tokens if not i in en_stop]
    # stem tokens
    #stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    # add tokens to list
    W_text.append(stopped_tokens)

In [57]:
#Turn our tokenized documents into a id <-> term dictionary
W_dict = corpora.Dictionary(W_text)
#Convert tokenized documents into a document-term matrix
W_corpus = [W_dict.doc2bow(text) for text in W_text]

In [60]:
# generate LDA model
W_ldamodel = gensim.models.ldamodel.LdaModel(W_corpus, num_topics=5, id2word = W_dict, passes=20)

In [61]:
print(W_ldamodel.print_topics(num_topics=5, num_words=10))

[(0, u'0.010*"experience" + 0.007*"walk" + 0.006*"staff" + 0.005*"recent" + 0.005*"area" + 0.005*"bar" + 0.005*"day" + 0.005*"service" + 0.005*"friendly" + 0.005*"check"'), (1, u'0.010*"ucla" + 0.009*"service" + 0.007*"staff" + 0.006*"first" + 0.006*"like" + 0.005*"one" + 0.005*"experience" + 0.005*"restaurants" + 0.005*"hills" + 0.004*"really"'), (2, u'0.000*"staff" + 0.000*"service" + 0.000*"front" + 0.000*"bar" + 0.000*"friendly" + 0.000*"desk" + 0.000*"one" + 0.000*"experience" + 0.000*"first" + 0.000*"business"'), (3, u'0.014*"staff" + 0.007*"service" + 0.007*"friendly" + 0.006*"check" + 0.005*"helpful" + 0.005*"best" + 0.005*"business" + 0.005*"desk" + 0.004*"front" + 0.004*"one"'), (4, u'0.008*"staff" + 0.008*"right" + 0.007*"service" + 0.007*"view" + 0.007*"lobby" + 0.006*"bar" + 0.006*"modern" + 0.005*"corner" + 0.005*"floor" + 0.005*"could"')]


In [51]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(ldamodel, W_corpus, W_dict)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      58.613554        1       1  0.049089  0.006177
4      14.370490        1       2  0.076553 -0.155042
5      14.285517        1       3  0.086341  0.075172
9      12.671017        1       4  0.126695  0.050425
8       0.009904        1       5 -0.056180  0.003883
3       0.009904        1       6 -0.056399  0.003882
6       0.009904        1       7 -0.056419  0.003918
2       0.009904        1       8 -0.056453  0.003876
7       0.009904        1       9 -0.056504  0.003855
1       0.009904        1      10 -0.056723  0.003853, topic_info=     Category       Freq           Term      Total  loglift  logprob
term                                                                
1289  Default  21.000000           pool  21.000000  30.0000  30.0000
845   Default  16.000000           year  16.000000  29.0000  29.0000
2261  Default  80.000000          staff  80.000000  28.0000  28.0000
1234  Default  14.000000           ucla  14.000000  27.0000  27.0000
297   Default  14.000000     convenient  14.000000  26.0000  26.0000
636   Default  34.000000           time  34.000000  25.0000  25.0000
518   Default  21.000000          right  21.000000  24.0000  24.0000
285   Default  22.000000         modern  22.000000  23.0000  23.0000
129   Default  21.000000        amazing  21.000000  22.0000  22.0000
247   Default  54.000000        service  54.000000  21.0000  21.0000
1342  Default  25.000000          small  25.000000  20.0000  20.0000
784   Default  18.000000        staying  18.000000  19.0000  19.0000
489   Default   7.000000          hills   7.000000  18.0000  18.0000
1172  Default  13.000000         around  13.000000  17.0000  17.0000
1128  Default  36.000000          check  36.000000  16.0000  16.0000
362   Default  32.000000            bar  32.000000  15.0000  15.0000
1211  Default  25.000000       business  25.000000  14.0000  14.0000
1497  Default  26.000000         always  26.000000  13.0000  13.0000
2388  Default  22.000000          lobby  22.000000  12.0000  12.0000
1058  Default  11.000000          quiet  11.000000  11.0000  11.0000
1365  Default  32.000000     experience  32.000000  10.0000  10.0000
1780  Default  13.000000          early  13.000000   9.0000   9.0000
774   Default  11.000000            spg  11.000000   8.0000   8.0000
347   Default   6.000000          bliss   6.000000   7.0000   7.0000
1101  Default   6.000000  accommodating   6.000000   6.0000   6.0000
1656  Default  18.000000    restaurants  18.000000   5.0000   5.0000
436   Default  10.000000           near  10.000000   4.0000   4.0000
884   Default  22.000000           view  22.000000   3.0000   3.0000
376   Default  27.000000           best  27.000000   2.0000   2.0000
218   Default  16.000000            got  16.000000   1.0000   1.0000
...       ...        ...            ...        ...      ...      ...
2261  Topic10   0.000278          staff  80.413092  -3.3565  -7.8317
1289  Topic10   0.000276           pool  21.583841  -2.0457  -7.8361
1958  Topic10   0.000276       friendly  41.114713  -2.6904  -7.8364
247   Topic10   0.000276        service  54.497896  -2.9727  -7.8368
129   Topic10   0.000276        amazing  21.784966  -2.0565  -7.8376
1128  Topic10   0.000276          check  36.128898  -2.5627  -7.8380
1365  Topic10   0.000276     experience  32.113119  -2.4449  -7.8380
636   Topic10   0.000276           time  34.283160  -2.5108  -7.8385
626   Topic10   0.000276        weekend  23.028076  -2.1129  -7.8385
1898  Topic10   0.000276             us  26.915023  -2.2691  -7.8387
362   Topic10   0.000276            bar  32.050176  -2.4438  -7.8389
893   Topic10   0.000276            one  31.316665  -2.4209  -7.8391
1211  Topic10   0.000276       business  25.109966  -2.2000  -7.8391
888   Topic10   0.000276          first  27.998589  -2.3090  -7.8391
376   Topic10   0.000276           best  27.593859  -2.2944  -7.83